In [1]:
import asyncio
import os
import tempfile
import urllib.request

from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.agents.azure._azure_ai_agent import AzureAIAgent
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import DefaultAzureCredential
import azure.ai.projects.models as models
from dotenv import load_dotenv
load_dotenv()
import yaml



In [ ]:
async def bing_example():
    # Ensure required environment variables are set.
    conn_str = os.getenv("AI_PROJECT_CONNECTION_STRING")
    bing_conn_name = os.getenv("BING_CONNECTION_NAME")
    if not conn_str:
        raise ValueError("The environment variable AI_PROJECT_CONNECTION_STRING is required.")
    if not bing_conn_name:
        raise ValueError("The environment variable BING_CONNECTION_NAME is required.")

    credential = DefaultAzureCredential()

    async with AIProjectClient.from_connection_string(  # type: ignore
        credential=credential, conn_str=conn_str
    ) as project_client:
        conn = await project_client.connections.get(connection_name=bing_conn_name)

        bing_tool = models.BingGroundingTool(conn.id)
        agent_with_bing_grounding = AzureAIAgent(
            name="AQ_bing_agent",
            description="An AI assistant with Bing grounding",
            project_client=project_client,
            deployment_name="gpt-4o",
            instructions="You are a helpful assistant.",
            tools=bing_tool.definitions,
            metadata={"source": "AzureAIAgent"},
        )

        result = await agent_with_bing_grounding.on_messages(
            messages=[TextMessage(content="What is Microsoft's annual leave policy?", source="user")],
            cancellation_token=CancellationToken(),
            message_limit=5,
        )
        #print(result)
        print(result.chat_message.content)

if __name__ == "__main__":
    await(bing_example())


In [ ]:

async def file_search_example():
    # Download README.md from GitHub
    readme_url = "https://raw.githubusercontent.com/microsoft/autogen/refs/heads/main/README.md"
    temp_file = None

    try:
        # Create a temporary file to store the downloaded README
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".md")
        urllib.request.urlretrieve(readme_url, temp_file.name)
        print(f"Downloaded README.md to {temp_file.name}")

        credential = DefaultAzureCredential()
        async with AIProjectClient.from_connection_string(  # type: ignore
            credential=credential, conn_str=os.getenv("AI_PROJECT_CONNECTION_STRING", "")
        ) as project_client:
            agent_with_file_search = AzureAIAgent(
                name="AQ_file_search_agent",
                description="An AI assistant with file search capabilities",
                project_client=project_client,
                deployment_name="gpt-4o",
                instructions="You are a helpful assistant.",
                tools=["file_search"],
                metadata={"source": "AzureAIAgent"},
            )

            ct: CancellationToken = CancellationToken()
            # Use the downloaded README file for file search
            await agent_with_file_search.on_upload_for_file_search(
                file_paths=[temp_file.name],
                vector_store_name="file_upload_index",
                vector_store_metadata={"source": "AzureAIAgent"},
                cancellation_token=ct,
            )
            result = await agent_with_file_search.on_messages(
                messages=[
                    TextMessage(content="Hello, what is AutoGen and what capabilities does it have?", source="user")
                ],
                cancellation_token=ct,
                message_limit=5,
            )
            print(result.chat_message.content)
    finally:
        # Clean up the temporary file
        if temp_file and os.path.exists(temp_file.name):
            os.unlink(temp_file.name)
            print(f"Removed temporary file {temp_file.name}")

if __name__ == "__main__":
    await(file_search_example())


In [6]:



async def code_interpreter_example():
    credential = DefaultAzureCredential()
    async with AIProjectClient.from_connection_string(  # type: ignore
        credential=credential, conn_str=os.getenv("AI_PROJECT_CONNECTION_STRING", "")
    ) as project_client:
        agent_with_code_interpreter = AzureAIAgent(
            name="code_interpreter_agent",
            description="An AI assistant with code interpreter capabilities",
            project_client=project_client,
            deployment_name="gpt-4o",
            instructions="You are a helpful assistant.",
            tools=["code_interpreter"],
            metadata={"source": "AzureAIAgent"},
        )

        await agent_with_code_interpreter.on_upload_for_code_interpreter(
            file_paths="./data/nifty_500_quarterly_results.csv",
            cancellation_token=CancellationToken(),
        )

        result = await agent_with_code_interpreter.on_messages(
            messages=[
                TextMessage(
                    content="Aggregate the number of stocks per industry and give me a markdown table as a result?",
                    source="user",
                )
            ],
            cancellation_token=CancellationToken(),
        )

        print(result.chat_message.content)


if __name__ == "__main__":

    await(code_interpreter_example())



Here is the markdown table showing the number of stocks per industry:

| Industry                            |   Number of Stocks |
|:------------------------------------|-------------------:|
| PHARMACEUTICALS                     |                 33 |
| BANKS                               |                 28 |
| AUTO PARTS & EQUIPMENT              |                 20 |
| IT CONSULTING & SOFTWARE            |                 19 |
| SPECIALTY CHEMICALS                 |                 18 |
| FINANCE (INCLUDING NBFCS)           |                 17 |
| ELECTRIC UTILITIES                  |                 14 |
| INTERNET SOFTWARE & SERVICES        |                 14 |
| CEMENT & CEMENT PRODUCTS            |                 13 |
| REALTY                              |                 11 |
| IRON & STEEL/INTERM.PRODUCTS        |                  9 |
| HEAVY ELECTRICAL EQUIPMENT          |                  9 |
| PERSONAL PRODUCTS                   |                  9 |
| INDUSTRIAL M